In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set()

## 分析用データフレームの作成

スクレイピングするデータ\
・場所・・・東京\
・期間・・・2010年～2021年\
・データの単位・・・週ごと

In [8]:
index = pd.date_range(start='2010-01-01', end='2022-12-31')

In [9]:
#データフレーム作成
df = pd.DataFrame(index=index)
df['temp'] = np.nan
df['hum'] = np.nan
df

,temp,hum
2010-01-01,NaN,NaN
2010-01-02,NaN,NaN
2010-01-03,NaN,NaN
2010-01-04,NaN,NaN
2010-01-05,NaN,NaN
...,...,...
2022-12-27,NaN,NaN
2022-12-28,NaN,NaN
2022-12-29,NaN,NaN
2022-12-30,NaN,NaN


## 株価の時系列データをスクレイピングする

In [10]:
year_month = df.groupby([df.index.year, df.index.month]).mean()
year_month.index

MultiIndex([(2010,  1),
            (2010,  2),
            (2010,  3),
            (2010,  4),
            (2010,  5),
            (2010,  6),
            (2010,  7),
            (2010,  8),
            (2010,  9),
            (2010, 10),
            ...
            (2022,  3),
            (2022,  4),
            (2022,  5),
            (2022,  6),
            (2022,  7),
            (2022,  8),
            (2022,  9),
            (2022, 10),
            (2022, 11),
            (2022, 12)],
           length=156)

In [13]:
for Y, m in year_month.index:
    #urlを更新
    url = f"https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year={Y}&month={m}&day=&view="
    #htmlのテーブル要素を取得
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    table = soup.find('table', {'class':'data2_s'})

    for iter, tr in enumerate(table.findAll('tr')):

        #最初の4行は処理を行わない
        if iter <= 3:
            continue
        #一列目と7列目と10列目の値を取得
        tds = tr.findAll('td')
        d = int(tds[0].string)
        try:
            _temp = float(tds[6].string)
            hum = float(tds[9].string)
        except:
            _temp, hum = [np.nan, np.nan]
        #データフレームに格納
        df.loc[pd.to_datetime(datetime.datetime(Y, m, d)), 'temp':'hum'] = [_temp, hum]

df

,temp,hum
2010-01-01,4.8,33.0
2010-01-02,6.3,35.0
2010-01-03,5.7,41.0
2010-01-04,6.5,41.0
2010-01-05,7.3,49.0
...,...,...
2022-12-27,6.8,48.0
2022-12-28,6.5,61.0
2022-12-29,8.1,51.0
2022-12-30,6.6,51.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4748 entries, 2010-01-01 to 2022-12-31
Freq: D
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    4732 non-null   float64
 1   hum     4732 non-null   float64
dtypes: float64(2)
memory usage: 240.3 KB


欠測値もなく正常にデータ収集することができた

In [16]:
df.to_csv('./temp&hum_2010_2022.csv')

## データの可視化

NameError: name 'plt' is not defined